In [1]:
%pip install llama-index
%pip install faiss-cpu
%pip install sentence-transformers
%pip install requests
%pip install python-dotenv

%pip install llama-index-embeddings-huggingface
%pip install llama-index-vector-stores-faiss 




[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgr

In [2]:
# 1. Load HTML File/s
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_dir="html").load_data()
print(f"Loaded {len(documents)} documents.")

# 2. Chunk with SentenceSplitter
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=2048, chunk_overlap=200)
nodes = splitter.get_nodes_from_documents(documents)
print(f"Generated {len(nodes)} chunks.")

# 3. Embed Chunks with HuggingFace
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")


Loaded 1 documents.
Generated 144 chunks.


/home/fynn/Coding/RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 4. Create Index
import faiss
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import VectorStoreIndex, StorageContext

faiss_index = faiss.IndexFlatL2(384)
vector_store = FaissVectorStore(faiss_index=faiss_index)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes,
    embed_model=embed_model,
    storage_context=storage_context,
)

# Persist index to disk
index.storage_context.persist(persist_dir="./faiss_index")
print("Index persisted to ./faiss_index")

In [ ]:
# 5. Define LLM

from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("KISSKI_URL")

if not api_key or not base_url:
    raise ValueError("Missing KISSKI_API_KEY or KISSKI_URL in environment.")

client = OpenAI(
    api_key=api_key,
    base_url=base_url
)

def ask_openai_llm(prompt: str) -> str:
    response = client.chat.completions.create(
        model="meta-llama-3.1-8b-instruct",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content


In [ ]:
# 6. Ask a Question
import textwrap
from IPython.display import Markdown, display

while True:
    query = input("🔍 Enter your question (or type 'exit'): ").strip()
    if query.lower() == 'exit':
        print("👋 Session ended.")
        break

    nodes = index.as_retriever().retrieve(query)
    context = "\n---\n".join([n.get_content() for n in nodes])
    full_prompt = f"""You are a helpful assistant. Use the following context to answer the question.

Context:
{context}

Question:
{query}"""

    answer = ask_openai_llm(full_prompt)
    print(f"\nQ:\n{query}")
    print("\nA:\n")
    display(Markdown(textwrap.dedent(answer)))